In [16]:
import os,sys
import pandas as pd
import pickle
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from wer import calculate_wer

from Models import cnn_rnn_model,simple_rnn,model_2,bidirectional_rnn_model_gpu,bidirectional_rnn_model
from Batch import Batch


In [3]:
data=pd.read_json('../artifacts/meta.json')
data.drop(data[data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/features.pkl','rb') as tel:
    features=pickle.load(tel)
    tel.close()

In [4]:
valid_data=pd.read_json("../artifacts/valid_meta.json")
valid_data.drop(valid_data[valid_data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/valid_features.pkl','rb') as val:
    valid_features=pickle.load(val)
    val.close()

In [5]:
FEATURES_LIST=features

TEXT_LIST=data["text"].to_numpy()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()
MINI_BATCH_SIZE=250


audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [17]:
bidirectional_rnn=bidirectional_rnn_model_gpu(input_dim=26, 
                        units=250)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 500)         417000    
_________________________________________________________________
norm_bidir_rnn (BatchNormali (None, None, 500)         2000      
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 29)          14529     
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0         
Total params: 433,529
Trainable params: 432,529
Non-trainable params: 1,000
_________________________________________________________________
None


In [7]:
cnn_rnn = cnn_rnn_model(input_dim=26, # change to 13 if you would like to use MFCC features
                        filters=250,
                        kernel_size=4, 
                        conv_stride=1,
                        conv_border_mode='same',
                        units=200,output_dim=29)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 250)         26250     
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 250)         1000      
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         90200     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 200)         800       
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 29)          5829      
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0   

In [8]:
deep_speech = model_2(input_dim=26,
                filters=100,
                kernel_size=4, 
                conv_stride=1,
                conv_border_mode='valid',
                units=250,
                activation='tanh',
                dropout_rate=0.2,
                number_of_layers=2,
                output_dim=29)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
layer_1_conv (Conv1D)        (None, None, 100)         10500     
_________________________________________________________________
conv_batch_norm (BatchNormal (None, None, 100)         400       
_________________________________________________________________
rnn_1 (GRU)                  (None, None, 250)         264000    
_________________________________________________________________
bt_rnn_1 (BatchNormalization (None, None, 250)         1000      
_________________________________________________________________
final_layer_of_rnn (GRU)     (None, None, 250)         376500    
_________________________________________________________________
bt_rnn_final (BatchNormaliza (None, None, 250)         1000

In [11]:
with open('models\\bidirectional_rnn_gpu.h5', 'rb') as t:
        bidirectional_rnn_model_hist=pickle.load( t)

In [35]:
with open('models\\deep_speech.pickle', 'rb') as f:
        deep_speech_hist=pickle.load( f)

In [45]:
with open('models\\simple_rnn_model.pickle', 'rb') as g:
        cnn_hist=pickle.load( g)

In [47]:
cnn_rnn.load_weights('./model/cnn.h5')

In [54]:
deep_speech.load_weights('./model/deep.h5')

In [49]:
from wer import predict

In [56]:
predict(audio_gen,35,'validation',deep_speech)

Truth: familia za waliopoteza maisha nchini cambodia baada ya kukanyagana katika tamasha la kijadi
Predicted: familiyatawliokodeza maisha njini kambodiabada a kukanya dala katikatamasha la idiali
wer: 21


21

In [53]:
predict(audio_gen,35,'validation',cnn_rnn)

Truth: familia za waliopoteza maisha nchini cambodia baada ya kukanyagana katika tamasha la kijadi
Predicted: kamila waliow deda mesha nchiniepambod a bada upana dalna katika tamashalakigaji
wer: 28


28

In [ ]:
predict(audio_gen,17,'validation',deep_speech)